In [1]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils
from function_util import *
from IMP import IMP
from SMP1 import SMP1
from SMP2 import SMP2
from FIS1 import FIS1
from FIS2 import FIS2
from FIS3 import FIS3

In [27]:
# load all the data 
gdf_utm = pd.read_pickle('gdf_utm.pkl')
edges_utm = pd.read_pickle('edges_utm.pkl')
nodes_utm = pd.read_pickle('nodes_utm.pkl')

# select error bound 
err_size = 38
# select cut off point for smp1, for use 60 following Quddus paper
cutoff = 60 

# initialize matched trajectory list
matched_point_list = []
# initialize matched edge list 
matched_edge_list = []
# initialize SMP1 result list 
res_list =[]
# initialize counter for matched_list 
count = 0

# Run IMP Algorithm to find first point 
point_index, curr_edge = IMP(gdf_utm, edges_utm, 2)

#store current edge 
matched_edge_list.append(curr_edge)

# matched current location with current edge 
curr_loc = gdf_utm.iloc[[point_index]]
matched_point = point_matching(curr_loc, curr_edge.iloc[0])
# find longitude and latitude for last matched data
matched_point['lon_lat'] = matched_point.to_crs({'init': 'epsg:4326'})
# store matched point 
matched_point_list.append(matched_point)

# move to the next iteration with for loop
start_iter = point_index + 1

for point_index in range(start_iter, len(gdf_utm)):    
    print(point_index)
    # update last location 
    prev_loc = gdf_utm.iloc[[point_index - 1]]

    # update current location 
    curr_loc = gdf_utm.iloc[[point_index]]
    
    # update previous matched position
    last_matched = matched_point_list[count]

    # perform SMP1 
    res = SMP1(curr_loc, curr_edge, prev_loc, last_matched, nodes_utm, edges_utm, gdf_utm)
    # store result list
    res_list.append(res)

    # decide whether vehicle is entering junction or not cut off 
    if res >= cutoff:
        # vehicle is not entering a new edge, matched edge doesnt change
        # update matched edge list 
        matched_edge_list.append(curr_edge)
    elif res < cutoff:
        # SMP1 detect vehicle is entering new edge, perform SMP 2 to update current edge
        curr_edge = SMP2(curr_loc, curr_edge, prev_loc, last_matched, err_size, nodes_utm, edges_utm, gdf_utm)
        if curr_edge is None:
            print(['No edge can be matched, stopped at trajectory number', point_index])
            break
        else:
            # there is no more matched edge, break the loop 
            matched_edge_list.append(curr_edge)
            
    # update matched point
    matched_point = point_matching(curr_loc, curr_edge.iloc[0])
    # find longitude and latitude for last matched data
    matched_point['lon_lat'] = matched_point.to_crs({'init': 'epsg:4326'})
    # store matched point 
    matched_point_list.append(matched_point)
    # update count 
    count +=  1

C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Pl

['no edeges intersects with error bound at iteration number', 1]
['no edeges intersects with error bound at iteration number', 2]
['no edeges intersects with error bound at iteration number', 3]
['no edeges intersects with error bound at iteration number', 4]
['no edeges intersects with error bound at iteration number', 5]


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Pl

['no edeges intersects with error bound at iteration number', 6]
['no edeges intersects with error bound at iteration number', 7]
['no edeges intersects with error bound at iteration number', 8]
['no edeges intersects with error bound at iteration number', 9]
['no edeges intersects with error bound at iteration number', 10]
['no edeges intersects with error bound at iteration number', 11]


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\U

...

C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\skfuzzy\membership\generatemf.py:340: RuntimeWarning: overflow encountered in exp
  return 1. / (1. + np.exp(- c * (x - b)))
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\skfuzzy\membership\generatemf.py:340: RuntimeWarning: overflow encountered in exp
  return 1. / (1. + np.exp(- c * (x - b)))
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could n

53
54
55
56


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\skfuzzy\membership\generatemf.py:340: RuntimeWarning: overflow encountered in exp
  return 1. / (1. + np.exp(- c * (x - b)))
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\skfuzzy\membership\generatemf.py:340: RuntimeWarning: overflow encountered in exp
  return 1. / (1. + np.exp(- c * (x - b)))
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When makin

57
58
['No edge can be matched, stop at trajectory number', 58]


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\skfuzzy\membership\generatemf.py:340: RuntimeWarning: overflow encountered in exp
  return 1. / (1. + np.exp(- c * (x - b)))
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\skfuzzy\membership\generatemf.py:340: RuntimeWarning: overflow encountered in exp
  return 1. / (1. + np.exp(- c * (x - b)))
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could n

In [18]:
matched_edge_df = pd.concat(matched_edge_list)
matched_point_df = pd.concat(matched_point_list)

# Visualize Result 

In [28]:
%matplotlib tk
# This is how we  visualize edges and error bound 

# plotting edges and starting point together 
f, ax = plt.subplots()

# location for all point
#locs_utm.plot(ax=ax)
point_locs = gdf_utm['geometry'].to_frame()
point_locs.plot(ax = ax)


# this plot all the road system 
edges_utm.plot(ax=ax)
# candidate_link.iloc[[0]].plot(ax = ax, color = 'Red')
# candidate_link.iloc[[3]].plot(ax = ax, color = 'Yellow')
# candidate_link.iloc[4:6].plot(ax = ax, color = 'Blue')
# candidate_link.iloc[6:8].plot(ax = ax)

# this plot the selected edge at time point 

# matched point plot
matched_point_df.plot(ax = ax, color = "Green")
# next_point_matching.plot(ax = ax, color = "Green")

# matched_edge 
matched_edge_df.plot(ax = ax, color = "Black")

# # plot closest node
# closest_node.plot(ax = ax, color = "Black")
# closest_edge.plot(ax = ax , color = "Black")
# # debuging for djiksta shortest path
# origin_node.plot(ax = ax, color = "Black")
# target_node.plot(ax = ax , color = "Black")

<Axes: >